# Tuning the model

Place all scalings of the features into the model, then use stepwise_selection() and calculate_vif() to determine the best features to include in our model


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.tools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, MaxAbsScaler, PowerTransformer, Normalizer, MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor


All of these scaling methods add a column to the dataframe for each feature they scale. The original columns are all kept in.
The workings of the functions are explained by comments in only the first function robust_scaling(), as each function has similar syntax and functionality

In [ ]:
def robust_scaling(df):

    possible_scalings =['adult_mortality', 'hepatitis_b', 'measles', 'bmi', 'under_five_deaths', 'polio',
                        'diphtheria', 'gdp_per_capita', 'population_mln', 'thinness_ten_nineteen_years',
                        'thinness_five_nine_years']

    #prevents the function from trying to scale features not in df
    features = [feature for feature in possible_scalings if feature in list(df.columns)]


    df_rob = df[features]
    #turn list elements to floats
    df_rob = df_rob.astype(float)

    #create scalar object and fit it to features
    rob = RobustScaler()
    rob.fit(df_rob)

    #transform the features into a new DataFrame df_scaled
    df_scaled = pd.DataFrame(rob.transform(df_rob),columns=[col + '_rob' for col in df_rob.columns])

    #join the scaled df to the original df at the correct index
    df_scaled['index'] = df.index
    df_scaled = df_scaled.join(df, on='index')
    df_scaled.set_index('index', inplace=True)

    return df_scaled

In [ ]:
def max_abs(df):

    max_abs = MaxAbsScaler()

    maxabs_columns = df[['adult_mortality', 'incidents_hiv', 'gdp_per_capita', 'population_mln']].copy()

    max_abs.fit(maxabs_columns)

    df_scaled_ma = pd.DataFrame(max_abs.transform(maxabs_columns), columns=[col + '_ma' for col in maxabs_columns.columns])

    df_scaled_ma['index'] = df.index

    df_scaled_ma = df_scaled_ma.join(df, on='index')
    df_scaled_ma.set_index('index', inplace=True)

    return df_scaled_ma

In [ ]:
def power_transform(df):

    pt = PowerTransformer()

    pt_columns = df[['infant_deaths', 'under_five_deaths', 'adult_mortality', 'alcohol_consumption', 'hepatitis_b', 'measles', 'polio', 'diphtheria', 'incidents_hiv', 'gdp_per_capita', 'population_mln', 'thinness_ten_nineteen_years', 'thinness_five_nine_years', 'economy_status_developed', 'economy_status_developing']].copy()
    pt.fit(pt_columns)

    df_scaled_pt = pd.DataFrame(pt.transform(pt_columns), columns=[col + '_pt' for col in pt_columns.columns])

    df_scaled_pt['index'] = df.index

    df_scaled_pt = df_scaled_pt.join(df, on='index')
    df_scaled_pt.set_index('index', inplace=True)

    return df_scaled_pt

In [ ]:
def normaliser(df):
    normalizer = Normalizer()

    norm_cols = df[['year', 'infant_deaths', 'under_five_deaths','adult_mortality', 'alcohol_consumption', 'hepatitis_b', 'measles','bmi', 'polio', 'diphtheria', 'incidents_hiv', 'gdp_per_capita',
                    'population_mln', 'thinness_ten_nineteen_years','thinness_five_nine_years', 'schooling', 'economy_status_developed','economy_status_developing']].copy()

    normalizer.fit(norm_cols)
    df_norm_scale = pd.DataFrame(normalizer.transform(norm_cols), columns=[columns + '_normed' for columns in norm_cols.columns])

    df_norm_scale['index'] = df.index

    df_norm_scale = df_norm_scale.join(df, on='index')
    df_norm_scale.set_index('index', inplace=True)

    return df_norm_scale

In [ ]:
def min_max(df):

    mm_scaler = MinMaxScaler()

    mm_cols = df[['year', 'infant_deaths', 'under_five_deaths','adult_mortality', 'alcohol_consumption', 'hepatitis_b', 'measles','bmi', 'polio', 'diphtheria', 'incidents_hiv', 'gdp_per_capita',
                  'population_mln', 'thinness_ten_nineteen_years','thinness_five_nine_years', 'schooling', 'economy_status_developed','economy_status_developing']].copy()

    mm_scaler.fit(mm_cols)

    df_min_max = pd.DataFrame(mm_scaler.transform(mm_cols), columns=[columns + '_min_max' for columns in mm_cols.columns])

    df_min_max['index'] = df.index

    df_min_max = df_min_max.join(df, on='index')
    df_min_max.set_index('index', inplace=True)

    return df_min_max

In [ ]:
def cleaning(df):

    # Making all columns be formatted the same (stripped, lowercase, spacing, underscores in the middle)
    clean_cols = list(df.columns)
    new_cols = []
    for col in clean_cols:
        new_cols.append(col.strip().replace('  ', ' ').replace(' ', '').lower())
    df.columns = new_cols

    # implementing some mapping and cleaning of nulls
    # People cannot survive with BMI's of <10 or >50. To believe that that would represent an entire population is wrong.
    df['bmi'] = df.apply(lambda x: np.nan if (x.bmi < 10 or x.bmi > 50) else x.bmi, axis=1)

    # 0 infant deaths for a country in a year is unrealistic
    df['infant_deaths'] = df['infant_deaths'].replace(0, np.nan)

    # 0 under-five deaths for a country in a year is also unrealistic
    df['under_five_deaths'] = df['under_five_deaths'].replace(0, np.nan)

    return df

In [ ]:
def stepwise_selection(X, y, threshold_in = 0.01, threshold_out = 0.05, verbose = True):
    # The function is checking for p-values (whether features are statistically significant) - lower is better
    included = [] # this is going to be the list of features we keep
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index = excluded, dtype = 'float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        # we add the feature with the lowest (best) p-value under the threshold to our 'included' list
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval)) # specifying the verbose text


        # backward step: removing features if new features added to the list make them statistically insignificant
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()

        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        # if the p-value exceeds the upper threshold, the feature will be dropped from the 'included' list
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [ ]:
def step_testing():

    df = pd.read_csv("sample_data/Life_Expectancy_Data_Updated.csv")

    features = ['year', 'infant_deaths', 'under_five_deaths',
                 'adult_mortality', 'alcohol_consumption', 'hepatitis_b', 'measles',
                 'bmi', 'polio', 'diphtheria', 'incidents_hiv', 'gdp_per_capita',
                 'population_mln', 'thinness_ten_nineteen_years',
                 'thinness_five_nine_years', 'schooling', 'economy_status_developed',
                 'economy_status_developing']

    df_clean = cleaning(df)

    X_train, X_test, y_train, y_test = train_test_split(df[features],df['life_expectancy'],test_size = 0.2, random_state = 63)
    #-----------------------------------------------

    #add columns for each scaling
    X_train_fe = robust_scaling(X_train)
    X_train_fe = max_abs(X_train_fe)
    X_train_fe = power_transform(X_train_fe)
    X_train_fe = normaliser(X_train_fe)
    X_train_fe = min_max(X_train_fe)

    X_test_fe = robust_scaling(X_test)


    #----------------------------------------------
    results = stepwise_selection(X_train_fe, y_train)


    #run model with rows outputted from stepwise
    X_train_fe = sm.add_constant(X_train_fe[results])
    lin_reg = sm.OLS(y_train, X_train_fe)
    model = lin_reg.fit()
    print(model.summary())
    y_pred = model.predict(X_train_fe)
    rmse = statsmodels.tools.eval_measures.rmse(y_train, y_pred)
    print(f"RMSE value of {rmse}")


    pass

In [ ]:
# calling the above function

step_testing()

In [ ]:
def calculate_vif(X, thresh = 5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        # this bit uses list comprehension to gather all the VIF values of the different variables
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif)) # getting the index of the highest VIF value
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc] # we delete the highest VIF value on condition that it's higher than the threshold
            dropped = True # if we deleted anything, we set the 'dropped' value to True to stay in the while loop

    print('Remaining variables:')
    print(X.columns[variables]) # finally, we print the variables that are still in our set
    print(vif)
    return X.iloc[:, variables] # and return our X cut down to the remaining variables

In [ ]:
def vif_testing():
    df = pd.read_csv("sample_data/Life_Expectancy_Data_Updated.csv")

    features = ['year', 'infant_deaths', 'under_five_deaths',
                'adult_mortality', 'alcohol_consumption', 'hepatitis_b', 'measles',
                'bmi', 'polio', 'diphtheria', 'incidents_hiv', 'gdp_per_capita',
                'population_mln', 'thinness_ten_nineteen_years',
                'thinness_five_nine_years', 'schooling', 'economy_status_developed',
                'economy_status_developing']

    df_clean = cleaning(df)

    X_train, X_test, y_train, y_test = train_test_split(df[features],df['life_expectancy'],test_size = 0.2, random_state = 63)
    #-----------------------------------------------

    #add specific scaling methods
    #output X_train_fe, X_test_fe
    X_train_fe = X_train
    X_train_fe = robust_scaling(X_train)
    X_train_fe = max_abs(X_train_fe)
    X_train_fe = power_transform(X_train_fe)
    X_train_fe = normaliser(X_train_fe)
    X_train_fe = min_max(X_train_fe)

    X_test_fe = robust_scaling(X_test)


    #----------------------------------------------
    X_train_vif = calculate_vif(X_train_fe)


    #run model with rows outputted from stepwise

    X_train_fe = sm.add_constant(X_train_vif)

    lin_reg = sm.OLS(y_train, X_train_fe)
    model = lin_reg.fit()
    print(model.summary())
    y_pred = model.predict(X_train_fe)
    rmse = statsmodels.tools.eval_measures.rmse(y_train, y_pred)
    print(f"RMSE value of {rmse}")

In [ ]:
# calling the function

vif_testing()


C:\Users\James\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


dropping 'year_min_max' at index: 0
dropping 'infant_deaths_min_max' at index: 0
dropping 'under_five_deaths_min_max' at index: 0
dropping 'adult_mortality_min_max' at index: 0
dropping 'alcohol_consumption_min_max' at index: 0
dropping 'hepatitis_b_min_max' at index: 0
dropping 'measles_min_max' at index: 0
dropping 'bmi_min_max' at index: 0
dropping 'polio_min_max' at index: 0
dropping 'diphtheria_min_max' at index: 0
dropping 'incidents_hiv_min_max' at index: 0
dropping 'gdp_per_capita_min_max' at index: 0
dropping 'population_mln_min_max' at index: 0
dropping 'thinness_ten_nineteen_years_min_max' at index: 0
dropping 'thinness_five_nine_years_min_max' at index: 0
dropping 'schooling_min_max' at index: 0
dropping 'economy_status_developed_min_max' at index: 0
dropping 'economy_status_developing_min_max' at index: 0
dropping 'economy_status_developed_pt' at index: 31
dropping 'economy_status_developing_pt' at index: 31
dropping 'adult_mortality_ma' at index: 31
dropping 'incidents_hi

C:\Users\James\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [ ]:
df = pd.read_csv("sample_data/Life_Expectancy_Data_Updated.csv")

features = [ 'under_five_deaths', 'alcohol_consumption', 'incidents_hiv', 'population_mln', 'economy_status_developed']

rmse_dict = {'empty' : 0}

print(rmse_dict)
for feature_dropped in features:

    print(f"Drop: {feature_dropped}")
    df_clean = cleaning(df)

    X = df_clean[features].drop(columns=feature_dropped).copy()
    y = df_clean['life_expectancy']

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 63)


    X_train_fe = X_train
    X_train_fe = sm.add_constant(X_train_fe)

    lin_reg = sm.OLS(y_train, X_train_fe)
    model = lin_reg.fit()
    y_pred = model.predict(X_train_fe)
    rmse = statsmodels.tools.eval_measures.rmse(y_train, y_pred)
    print(f"RMSE value of {rmse}")

    rmse_dict[feature_dropped] = rmse


print(rmse_dict)

## Streamlining the model

In [ ]:
def power_transform(df, scale_columns = ['adult_mortality','polio','incidents_hiv','thinness_five_nine_years','alcohol_consumption','measles']):

    pt = PowerTransformer()

    #probaby not necessary
    scale_columns = [col for col in scale_columns if col in ['adult_mortality','polio','incidents_hiv','thinness_five_nine_years','alcohol_consumption','measles']]

    pt_columns = df[scale_columns].copy()

    df.drop(columns=scale_columns, inplace=True)

    pt.fit(pt_columns)

    df_scaled_pt = pd.DataFrame(pt.transform(pt_columns), columns=[col + '_pt' for col in pt_columns.columns])

    df_scaled_pt['index'] = df.index


    df_scaled_pt = df_scaled_pt.join(df, on='index')
    df_scaled_pt.set_index('index', inplace=True)

    return df_scaled_pt

In [ ]:
def normaliser(df, scale_columns = ['thinness_ten_nineteen_years', 'schooling']):
    normalizer = Normalizer()

    #probably not necessary
    scale_columns = [col for col in scale_columns if col in ['thinness_ten_nineteen_years', 'schooling']]

    norm_cols = df[scale_columns].copy()

    df.drop(columns=scale_columns, inplace=True)

    normalizer.fit(norm_cols)
    df_norm_scale = pd.DataFrame(normalizer.transform(norm_cols), columns=[columns + '_normed' for columns in norm_cols.columns])

    df_norm_scale['index'] = df.index

    df_norm_scale = df_norm_scale.join(df, on='index')
    df_norm_scale.set_index('index', inplace=True)

    return df_norm_scale

In [ ]:
def cleaning_detailed(df):

    # Making all columns be formatted the same (stripped, lowercase, spacing, underscores in the middle)
    clean_cols = list(df.columns)
    new_cols = []
    for col in clean_cols:
        new_cols.append(col.strip().replace('  ', ' ').replace(' ', '').lower())
    df.columns = new_cols

    return df

In [ ]:
def detailed_model():

    features = ['adult_mortality', 'alcohol_consumption', 'measles',
                'polio', 'incidents_hiv',
                'thinness_ten_nineteen_years',
                'thinness_five_nine_years', 'schooling', 'economy_status_developed']

    df = pd.read_csv('Life-Expectancy-Data-Updated.csv')

    df_clean = cleaning_detailed(df)

    X = df[features]
    y = df['life_expectancy']

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 63)

    X_train_clean = cleaning_detailed(X_train)
    X_test_clean = cleaning_detailed(X_test)

    #transform training data
    X_train_fe = power_transform(X_train_clean)
    X_train_fe = normaliser(X_train_fe)

    #transform testing data
    X_test_fe = power_transform(X_test_clean)
    X_test_fe = normaliser(X_test_fe)

    X_train_fe = sm.add_constant(X_train_fe)
    X_test_fe = sm.add_constant(X_test_fe)

    lin_reg = sm.OLS(y_train, X_train_fe)
    model = lin_reg.fit()
    print(model.summary())
    y_pred = model.predict(X_train_fe)
    #
    rmse = statsmodels.tools.eval_measures.rmse(y_train, y_pred)
    print(f"RMSE value of {rmse}")
    # test on test?
    # #print(X_test_fe.shape)
    # y_pred_test = model.predict(X_test_fe)
    #
    # test_rmse = statsmodels.tools.eval_measures.rmse(y_test, y_pred_test)
    #
    # print(f"Test RMSE of {test_rmse}")

    pass

In [ ]:
detailed_model()

                            OLS Regression Results                            
Dep. Variable:        life_expectancy   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                     3467.
Date:                Mon, 10 Jul 2023   Prob (F-statistic):               0.00
Time:                        15:30:22   Log-Likelihood:                -5313.5
No. Observations:                2291   AIC:                         1.065e+04
Df Residuals:                    2281   BIC:                         1.070e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
cons

C:\Users\James\AppData\Local\Temp/ipykernel_29704/3110962019.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=scale_columns, inplace=True)
C:\Users\James\AppData\Local\Temp/ipykernel_29704/3110962019.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=scale_columns, inplace=True)
C:\Users\James\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)
C:\Users\James\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: Fut

## Notes on dropping

* population_mln had a  p-value of 0.884 so was removed
* hepatitis_b had a p-value of 0.381 so was removed
* Condition Number (started at ~500,000)
    * gdp_per_capita gave us high condition number in previous tests (removing this feature brought this number down to 43.5)